In [1]:
import serial
import time
import keyboard

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

In [2]:
# connect with Arduino

ser = serial.Serial('COM3', 9600, timeout=1)
time.sleep(1)

In [3]:
# read write the serial connection at a given sampling rate

def write_read(x):
    ser.write(bytes(x, 'utf-8'))
    time.sleep(0.01)
    data = ser.readline()
    return data

In [4]:
# start to control the Arduino while collecting the data

values=[]
timestamp=[]
i= True
num = '0'
print("Press <1> to switch power on")
print("Press <0> to switch power off")
print("Press <q> to quit")
print(".............................")

while i:  # making a loop
    
    if keyboard.is_pressed('q'):  # if key 'q' is pressed 
        print('')
        print('You Pressed <q>uit!')
        i = False                 # finishing the loop
        
    elif keyboard.is_pressed('0'): # if key '0' is pressed
        num = '0'
        
    elif keyboard.is_pressed('1'): # if key '1' is pressed
        num = '1'
        
    else:
        value = write_read(num)  # if user pressed a key other than the given key the loop will break
        timestamp.append(time.time())
        dummy = value.decode().rstrip().split()
       # dum0 = np.array([[int(j)/1024*5 for j in dummy[1:]]])
        print(pd.to_datetime(time.time(), unit= 's'),dummy, 
              #dum0[0],
              '            ', end = "\r")
        values.append(dummy)

ser.close()

Press <1> to switch power on
Press <0> to switch power off
Press <q> to quit
.............................
2022-11-17 07:46:37.012257280 ['0', '0', '0']                   
You Pressed <q>uit!


# 

In [5]:
#timestamp[len(timestamp)-1]-timestamp[0]
df = pd.DataFrame(values, index = pd.to_datetime(timestamp, unit = 's'), columns = ['S','V0','V1'])
df["S"]  = pd.to_numeric(df["S"],  errors='coerce', downcast="float")
df["V0"] = pd.to_numeric(df["V0"], errors='coerce', downcast="float")
df["V1"] = pd.to_numeric(df["V1"], errors='coerce', downcast="float")

df["V0"] = df["V0"]/1024*5
df["V1"] = df["V1"]/1024*5

df.describe()

,S,V0,V1
count,251.000000,251.000000,251.000000
mean,0.565737,1.854535,0.979364
std,0.496650,2.059785,1.473072
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,0.600586,0.000000
75%,1.000000,3.334961,1.484375
max,1.000000,4.995117,4.995117


In [6]:
#df.to_excel("C:/Users/Admin/Pythonprojects/RAMS/data/kanweg.xlsx")
df.to_excel("C:/Users/Admin/Pythonprojects/RAMS/data/Arduino_trainingset_3Vars.xlsx")

# 